In [ ]:
# default_exp chem

# Chem

> Core comp chem functions implemented with RDKit

In [ ]:
#hide
from nbdev.showdoc import *
%load_ext autoreload
%autoreload 2

In [ ]:
# export
# hide
from mrl.imports import *
from mrl.core import *
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, rdMolDescriptors, Descriptors, rdMMPA
from rdkit.Chem.FilterCatalog import *

//anaconda3/envs/insight/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: to-Python converter for boost::shared_ptr<RDKit::FilterCatalogEntry const> already registered; second conversion method ignored.
  return f(*args, **kwds)


## RDKit i/o

`to_mol` and `to_smile` are functions that make it easy to work with both SMILES strings and RDKit `Mol` objects. For example, if a function requires a `mol` input, adding `mol = to_mol(mol)` allows the function to take either SMILES strings or RDKit mols as input

In [ ]:
# export
def to_mol(smile_or_mol):
    if type(smile_or_mol) == str:
        mol = Chem.MolFromSmiles(smile_or_mol)
    else:
        mol = smile_or_mol
        
    return mol

def to_smile(smile_or_mol):
    
    if type(smile_or_mol)==Chem.Mol:
        smile = Chem.MolToSmiles(smile_or_mol)
    else:
        smile = smile_or_mol
        
    return smile

In [ ]:
assert type(to_smile('CCC')) == str
assert type(to_mol('CCC')) == Chem.Mol
assert type(to_smile(Chem.MolFromSmiles('CCC'))) == str
assert type(to_mol(Chem.MolFromSmiles('CCC'))) == Chem.Mol

## Mol Descriptors

These are a bunch of wrappers for standard RDKit functions. The reason for doing this is RDKit functions can't be pickled, which causes all sorts of problems for multiprocessing. This is fixed by creating a wrapper function

For example:

In [ ]:
try:
    output = maybe_parallel(rdMolDescriptors.CalcExactMolWt, [to_mol('CCC')])
    print('Parallel execution succeeded')
except:
    print('parallel execution failed')
    
def wrapper(mol):
    return rdMolDescriptors.CalcExactMolWt(mol)

try:
    output = maybe_parallel(wrapper, [to_mol('CCC')])
    print('Parallel execution succeeded')
except:
    print('parallel execution failed')

parallel execution failed
Parallel execution succeeded


Sadly, having a generic wrapper constructor also fails to pickle because such a wrapper requires an RDKit function as input to construct the wrapper, which brings back the pickle problems (see code example below). This leaves us with manually defining wrapper functions for RDKit functions

In [ ]:
def rdkit_wrapper(rdkit_func):
    def wrapper(mol):
        return rdkit_func(mol)
    
    return wrapper

try:
    output = maybe_parallel(rdkit_wrapper(rdMolDescriptors.CalcExactMolWt), [to_mol('CCC')])
    print('Parallel execution succeeded')
except:
    print('parallel execution failed')

parallel execution failed


In [ ]:
# export
def molwt(mol):
    return rdMolDescriptors.CalcExactMolWt(mol)

def hbd(mol):
    return rdMolDescriptors.CalcNumHBD(mol)

def hba(mol):
    return rdMolDescriptors.CalcNumHBA(mol)
    
def tpsa(mol):
    return rdMolDescriptors.CalcTPSA(mol)
    
def rotbond(mol):
    return rdMolDescriptors.CalcNumRotatableBonds(mol)
    
def fsp3(mol):
    return rdMolDescriptors.CalcFractionCSP3(mol)
    
def logp(mol):
    return Descriptors.MolLogP(mol)
    
def rings(mol):
    return rdMolDescriptors.CalcNumRings(mol)
    
def heteroatoms(mol):
    return rdMolDescriptors.CalcNumHeteroatoms(mol)

def aromaticrings(mol):
    return rdMolDescriptors.CalcNumAromaticRings(mol)

In [ ]:
try:
    _ = maybe_parallel(hbd, [to_mol('CCC')])
    output = 'success'
except:
    output = 'fail'
    
assert output == 'success'

## Substructure Matching

This class is used for substructure matching an input `Mol` against a list of SMARTS. 

__Note__: Substructure matching is tricky. Be sure to verify your SMARTS before putting a large number of them into a filter.

`CatalogMatch` functions as a base class to match `Mol` objects against any generic catalog. `has_match` will return a single boolean value for if the `Mol` matches one of he filters in the catalog. `get_matches` will return a list of bools for all elements in the catalog.

`SMARTSMatch` will generate a catalog from a list of SMARTS

`PAINSMatch`, `PAINSAMatch`, `PAINSBMatch` and `PAINSCMatch` specify different PAINS catalogs present in RDKit (see [here](https://www.rdkit.org/docs/source/rdkit.Chem.rdfiltercatalog.html#rdkit.Chem.rdfiltercatalog.FilterCatalogParams.FilterCatalogs))

In [ ]:
# export
class Catalog():
    '''
    Base Class for SMARTS matching
    
    Inputs: `catalog`, RDKit `FilterCatalog`
    '''
    def __init__(self, catalog):
        self.catalog = catalog
        self.names = [self.catalog.GetEntryWithIdx(i).GetDescription() 
                      for i in range(self.catalog.GetNumEntries())]
            
    def __call__(self, mol, criteria='any'): 
        assert criteria in ('any', 'all'), "criteria must be one of ('any', 'all')"
        if criteria=='all':
            output = maybe_parallel(self.get_matches, mol)
        else:
            output = maybe_parallel(self.has_match, mol)
            
        return output
        
    def has_match(self, mol):
        mol = to_mol(mol)
        return self.catalog.HasMatch(mol)
    
    def get_matches(self, mol):
        mol = to_mol(mol)
        matches = [i.GetDescription() for i in self.catalog.GetMatches(mol)]
        return [i in matches for i in self.names]
    
class SmartsCatalog(Catalog):
    '''
    Class for SMARTS matching
    
    Inputs: `smarts`, list of smarts
    '''
    def __init__(self, smarts):
        self.smarts = smarts
        catalog = FilterCatalog()
        for s in smarts:
            catalog.AddEntry(FilterCatalogEntry(s, SmartsMatcher(s,s)))
            
        super().__init__(catalog)
        
    def get_matches(self, mol):
        mol = to_mol(mol)
        matches = [i.GetDescription() for i in self.catalog.GetMatches(mol)]
        return [i in matches for i in self.smarts]
        
        
class ParamsCatalog(Catalog):
    '''
    Generates `CatalogMatch` object from `params`, a `FilterCatalogParams` object
    '''
    def __init__(self, params):
        p = FilterCatalogParams()
        p.AddCatalog(params)
        catalog = FilterCatalog(params)
        super().__init__(catalog)
        
class PAINSCatalog(ParamsCatalog):
    "Full PAINS filter matching"
    def __init__(self):
        super().__init__(FilterCatalogParams.FilterCatalogs.PAINS)
        
class PAINSACatalog(ParamsCatalog):
    "PAINS A filter matching"
    def __init__(self):
        super().__init__(FilterCatalogParams.FilterCatalogs.PAINS_A)
        
class PAINSBCatalog(ParamsCatalog):
    "PAINS B filter matching"
    def __init__(self):
        super().__init__(FilterCatalogParams.FilterCatalogs.PAINS_B)
        
class PAINSCCatalog(ParamsCatalog):
    "PAINS C filter matching"
    def __init__(self):
        super().__init__(FilterCatalogParams.FilterCatalogs.PAINS_C)

class ZINCCatalog(ParamsCatalog):
    "ZINC filter matching"
    def __init__(self):
        super().__init__(FilterCatalogParams.FilterCatalogs.ZINC)
        
class BRENKCatalog(ParamsCatalog):
    "BRENK filter matching"
    def __init__(self):
        super().__init__(FilterCatalogParams.FilterCatalogs.BRENK)

In [ ]:
smarts = [
    '[*]-[#6]1:[#6]:[#6](-[#0]):[#6]:[#6](-[*]):[#6]:1',
    '[*]-[#6]1:[#6]:[#6](-[*]):[#6]:[#6]:[#6]:1',
    '[*]-[#6]1:[#6]:[#6]:[#6]:[#6]:[#6]:1',
    '[*]-[#6]1:[#6]:[#6](-[#7]-[*]):[#6]:[#6]:[#6]:1',
    '[#6]1:[#6]:[#7]:[#6]:[#6]:[#6]:1'
]

sm = SmartsCatalog(smarts)

smiles = [
    'c1ccccc1',
    'Cc1cc(NC)ccc1',
    'Cc1cc(NC)cnc1',
    'Cc1cccc(NCc2ccccc2)c1'
]

mols = [to_mol(i) for i in smiles]

assert sm(mols) == [False, True, True, True]

## Fingerprints

This section deals with creating and manipulating molecular fingerprints. Below are functions for generating different forms of Morgan fingerprints (ECFP4, ECFP6, FCFP4, FCFP6). Fingerprints by default are generated as RDKit `ExplicitBitVect` objects, but can be converted to numpy arrays using the `fp_to_array` function.

Fingerprint similarity functions using Tanimoto, Dice and Cosine metrics are implemented for both `ExplicitBitVect` and `ndarray` objects.

__Note__ following cheminformatics convention, fingerprint metrics are implemented as _similarities_ rather than _distances_. The metrics used have the relationship `similarity = 1 - distance`. For using fingerprint difference metrics in machine learning applications, be sure you are using the correct relationship (similarity vs difference) for your task.

In [ ]:
# export

def morgan_fp(mol, radius=3, nbits=2048, use_features=False):
    "morgan fingerprint"
    mol = to_mol(mol)
    fp = rdMolDescriptors.GetMorganFingerprintAsBitVect(mol, radius, nBits=nbits, useFeatures=use_features)
    return fp

def ECFP4(mol):
    "ECFP4 Fingerprint"
    return morgan_fp(mol, radius=2, use_features=False)

def ECFP6(mol):
    "ECFP6 Fingerprint"
    return morgan_fp(mol, radius=3, use_features=False)

def FCFP4(mol):
    "FCFP4 Fingerprint"
    return morgan_fp(mol, radius=2, use_features=True)

def FCFP6(mol):
    "FCFP6 Fingerprint"
    return morgan_fp(mol, radius=3, use_features=True)

def fp_to_array(fp):
    "Converts RDKit `ExplicitBitVec` to numpy array"
    fp = np.asarray(fp, dtype=np.int8)

    return fp

def tanimoto(fps1, fps2):
    'Tanimoto similarity'
    return 1-distance.cdist(fps1, fps2, metric='jaccard')

def tanimoto_rd(fp, fps):
    return DataStructs.BulkTanimotoSimilarity(fp, fps)

def dice(fps1, fps2):
    'Dice similarity'
    return 1-distance.cdist(fps1, fps2, metric='dice')

def dice_rd(fp, fps):
    return DataStructs.BulkDiceSimilarity(fp, fps)

def cosine(fps1, fps2):
    'Cosine similarity'
    return 1-distance.cdist(fps1, fps2, metric='cosine')

def cosine_rd(fp, fps):
    return DataStructs.BulkCosineSimilarity(fp, fps)

When computing similarities between fingerprints, several things need to be lined up. Different methods are needed for different fingerprint formats (`ndarray` vs `ExplicitBitVect`) and different distance metrics.

The `FP` class holds logic to make this easy. 

The `FP.get_fingerprint` function allows for parallel processing of fingerprint generation.

The `FP.fingerprint_similaity` routes fingerprints to the correct similarity function based on the fingerprint's array type and the simillarity metric used.

For cases where instantiating a class isn't helpful, `get_fingerprint` and `fingerprint_similarities` work as functional wrappers around `FP`.

In [ ]:
# export

class FP():
    '''
    FP - class for manipulating molecular fingerprints
    
    fp = FP()
    mol_fp = fp.get_fingerprint(mol, 'ECFP6')
    '''
    def __init__(self):
        self.similarities = {
            'tanimoto' : {'rdkit' : tanimoto_rd,
                          'numpy' : tanimoto},
            'dice'     : {'rdkit' : dice_rd,
                          'numpy' : dice},
            'cosine'   : {'rdkit' : cosine_rd,
                          'numpy' : dice}
        }
        
        self.fps = {'ECFP4' : ECFP4,
                    'ECFP6' : ECFP6,
                    'FCFP4' : FCFP4,
                    'FCFP6' : FCFP6}
        
    def get_fingerprint(self, mol, fp_type='ECFP6', output_type='rdkit'):
        '''
        Generates fingerprint for `mol`.
        
        `mol` can either be a `Mol` object or a list of `Mol` objects
        '''
        method = self.fps[fp_type]

        fps = maybe_parallel(method, mol)

        if output_type == 'numpy':
            fps = fp_to_array(fps)

        return fps
    
    def fingerprint_similaity(self, fps1, fps2, metric):
        '''
        Computes the similarity between `fps1` and `fps2` using `metric`
        
        Inputs `fps1` and `fps2` can be any of the following:
            `ndarray` Numpy vector (ndim=1) single fingerprint
            `ndarray` Numpy matrix (ndim=2) of stacked fingerprints
            `ExplicitBitVect` RDKit explicit bit vec
            `list` of `ExplicitBitVect`
            
        Both `fps1` and `fps2` should have the same array type (`ndarray` or `ExplicitBitVec`)
        '''
        input_type = self._np_or_rd(fps1)
        
        metric = self.similarities[metric][input_type]
        
        if input_type == 'numpy':
            fps1 = self._maybe_stack(fps1)
            fps2 = self._maybe_stack(fps2)
            
            similarities = self._fp_similarity_numpy(fps1, fps2, metric)
            
        else:
            fps1 = self._maybe_list(fps1)
            fps2 = self._maybe_list(fps2)
        
            similarities = self._fp_similarity_rdkit(fps1, fps2, metric)
            
        return similarities
    
    def _maybe_stack(self, inp):
        "helper function to prepare numpy fingerprints"
        if type(inp)==list:
            inp = np.stack(inp)
            
        if inp.ndim == 1:
            inp = inp[None,:]
        
        return inp
    
    def _maybe_list(self, inp):
        "helper function to prepare rdkit fingerprints"
        if type(inp) == DataStructs.cDataStructs.ExplicitBitVect:
            inp = [inp]
            
        return inp
    
    def _fp_similarity_rdkit(self, fps1, fps2, metric):
        'FP similarity using RDKit methods'
        similarities = []

        for fp in fps1:
            dist = metric(fp, fps2)
            similarities.append(dist)

        return np.squeeze(np.array(similarities))
    
    def _fp_similarity_numpy(self, fps1, fps2, metric):
        'FP similarity using Numpy methods'
        return np.squeeze(metric(fps1, fps2))
    
    def _np_or_rd(self, inp):
        if type(inp) == np.ndarray:
            output = 'numpy'
        elif type(inp) == list:
            output = self._np_or_rd(inp[0])
        else:
            output = 'rdkit'

        return output
    
def get_fingerprint(mol, fp_type):
    fp = FP()
    return fps.get_fingerprint(mol, fp_type)

def fingerprint_similarities(fps1, fps2, metric):
    fp = FP()
    return fp.fingerprint_similarity(fps1, fps2, metric)

In [ ]:
fp = FP()
fps = fp.get_fingerprint(mols, fp_type='ECFP4', output_type='rdkit')
fps_np = fp_to_array(fps)

assert np.allclose(fp.fingerprint_similaity(fps, fps, 'tanimoto'), 
                   fp.fingerprint_similaity(fps_np, fps_np, 'tanimoto'))

## Mol Operations

Functions for editing or manipulating `Mol` objects.

`fuse_on_atom_mapping` and `fuse_on_link` convert molecular fragments into fused `Mol` objects. `fuse_on_atom_mapping` relies on RDKit's atom mapping convention (ie `[*:1]CC`), while `fuse_on_link` relies on user specified linkages.

In [ ]:
# export
def fuse_on_atom_mapping(fragment_string):
    '''
    Inputs: `fragment_string`, str or Chem.Mol
    
    Outputs: str
    
    Merges a series of molecular fragments into a single compound by atom mapping
    
    ie `R1-[*:1].R2-[*:1] >> R1-R2`
    
    Fragments with paired atom mappings will be fused on the atoms connected to the mapped dummies.
    Mappings that occur once or more than 2 times are ignored
    '''

    mol = to_mol(fragment_string)

    new_mol = Chem.RWMol(mol)

    maps = defaultdict(list)
    for atom in new_mol.GetAtoms():
        atom_map = atom.GetAtomMapNum()
        if atom_map and atom.GetAtomicNum()==0:
            maps[atom_map].append(atom)

    remove_idxs = []
    missing = []

    for map_num in maps.keys():
        atoms = maps[map_num]
        if len(atoms) == 2:
            atom1, atom2 = atoms

            neighbor1_idx = atom1.GetNeighbors()[0].GetIdx()
            neighbor2_idx = atom2.GetNeighbors()[0].GetIdx()
            atom1_idx = atom1.GetIdx()
            atom2_idx = atom2.GetIdx()
            bond_order = atom2.GetBonds()[0].GetBondType()

            new_mol.AddBond(neighbor1_idx,
                         neighbor2_idx,
                         order=bond_order)

            remove_idxs += [atom1_idx, atom2_idx]

        elif len(atoms) == 1:
            missing.append(map_num)

    for idx in sorted(remove_idxs)[::-1]:
        new_mol.RemoveAtom(idx)

    new_smile = Chem.MolToSmiles(new_mol.GetMol())

    return new_smile

In [ ]:
assert fuse_on_atom_mapping('[*:1]CC.[*:1]CC') == 'CCCC'
assert fuse_on_atom_mapping(to_mol('[*:1]CC.[*:1]CC')) == 'CCCC'
assert fuse_on_atom_mapping('[*:1]CC.[*:2]CC') == 'CC[*:1].CC[*:2]'

In [ ]:
# export
def fuse_on_link(fragment_string, links):
    '''
    Inputs: `fragment_string`, str or Chem.Mol, `links`, list of defines linkages
    
    Outputs: str
    
    Merges a series of molecular fragments into a single compound by `links`
    
    ie `fuse_on_link('R1-[Rb].R2-[Rb]', ['[Rb]']) >> 'R1-R2'`
    
    Fragments matching a given link are defined by substring searching. 
    Links that occur once or more than 2 times are ignored
    
    Note: inputs with RDKit atom mapping (ie `[*:1]CC`) will fail
    '''
    fragment_string = to_smile(fragment_string)
    fragments = fragment_string.split('.')
    
    for link in links:
        to_link = [i for i in fragments if link in i]
        
        if len(to_link)==2:
            reaction = AllChem.ReactionFromSmarts(f'[*:1]-{link}.{link}-[*:2]>>[*:1]-[*:2]')
            new_smiles = Chem.MolToSmiles(reaction.RunReactants([Chem.MolFromSmiles(i) for i in to_link])[0][0])
            fragments = [new_smiles] + [i for i in fragments if not i in to_link]
            
    fused = to_mol('.'.join(fragments))
    return to_smile(fused)

In [ ]:
assert fuse_on_link('[Rb]CC.[Rb]CC', ['[Rb]']) == 'CCCC'
assert fuse_on_link('[Rb]CC.[Rb]CC', ['[Pb]']) == 'CC[Rb].CC[Rb]'

In [ ]:
# hide
from nbdev.export import notebook2script; notebook2script()

Converted 00_core.ipynb.
Converted 01_chem.ipynb.
Converted 03_templates.ipynb.
Converted index.ipynb.
